This notebook is for merging qualtrics data that was cleaned and processed by Prof Leslie Synder to GA data.

In [2]:
import pandas as pd # Pandas is an open source library providing high-performance, easy-to-use data structures and data analysis tools for Python
import json # JSON encoder and decoder for Python
import re
from functools import reduce
import numpy as np
from math import isnan
import matplotlib.pyplot as plt
import operator
import datetime
import seaborn as sns
file_loc = "C:/Users/huixin/Dropbox/research/menoplan/beta_phase/cleaned_analysis/"

In [6]:
qualtrics_data = pd.read_csv("C:/Users/huixin/Dropbox/research/menoplan/beta_phase/qualtrics_data/qualtrics_02nov.csv")


In [152]:
ga_data = pd.read_csv("C:/Users/huixin/Dropbox/research/menoplan/beta_phase/ga_data/02nov_os.csv")
ga_data = ga_data.rename(columns={'IP Address': 'IPAddress', 'Avg. Session Duration': 'muSessionDuration'})
ga_data = ga_data.iloc[:,0:-3]
ga_data = ga_data.drop(columns="Bounce Rate")

In [153]:
ga_data['muSessionDuration'] = ga_data['muSessionDuration'].apply(pd.to_datetime, format='%H:%M:%S').apply(datetime.datetime.time)
ga_data['muSessionDuration'] = ga_data['muSessionDuration'].apply(lambda x: x.second + x.minute*60 + x.hour*3600)
ga_data

,Operating System,IPAddress,Users,New Users,Sessions,Pages / Session,muSessionDuration
0,Android,99.93.208.84,1.0,1.0,1.0,4.0,155
1,iOS,99.92.172.96,1.0,1.0,1.0,7.0,133
2,Android,99.89.44.176,1.0,1.0,1.0,4.0,185
3,Android,99.82.254.180,1.0,1.0,1.0,6.0,252
4,Windows,99.8.85.105,1.0,1.0,1.0,8.0,449
...,...,...,...,...,...,...,...
686,10/30/2021,0,NaN,NaN,NaN,NaN,0
687,10/31/2021,0,NaN,NaN,NaN,NaN,0
688,11/1/2021,2,NaN,NaN,NaN,NaN,0
689,11/2/2021,1,NaN,NaN,NaN,NaN,0


In [154]:
meanDurPerSession = ga_data.groupby(['IPAddress', 'Operating System'], as_index=False).agg({'muSessionDuration':'mean', 'Pages / Session': 'mean'})
meanDurPerSession.columns = meanDurPerSession.columns.str.strip('/?=')
meanDurPerSession

,IPAddress,Operating System,muSessionDuration,Pages / Session
0,0,10/29/2021,0.0,NaN
1,0,10/30/2021,0.0,NaN
2,0,10/31/2021,0.0,NaN
3,1,10/14/2021,0.0,NaN
4,1,10/25/2021,0.0,NaN
...,...,...,...,...
683,99.82.254.180,Android,252.0,6.0
684,99.89.44.176,Android,185.0,4.0
685,99.92.172.96,iOS,133.0,7.0
686,99.93.208.84,Android,155.0,4.0


In [159]:
meanDurPerSession = meanDurPerSession.groupby(['IPAddress', 'Operating System'])['muSessionDuration'].agg("mean").unstack(level=1)

In [160]:
r = re.compile('\d*([.,\/]?\d+)')# use regex to find dates
newlist = list(filter(r.match, ls)) # create list of dates 
meanDurPerSession = meanDurPerSession.drop(columns=newlist) #we do not want the dates

In [163]:
meanDurPerSession = meanDurPerSession.drop(columns="Day Index")

In [165]:
meanDurPerSession.fillna(0).loc['140.186.252.50']

Operating System
Android         0.0
Chrome OS       0.0
Linux           0.0
Macintosh       0.0
Windows         0.0
iOS          1780.0
Name: 140.186.252.50, dtype: float64

In [135]:
OS.to_csv(file_loc + "02nov_os_cleaned.csv")

In [104]:
pd.DataFrame(meanDurPerSession).unstack(level=1)

Operating System  IPAddress    
Android           0                  NaN
                  1                  NaN
                  1.127.30.239       NaN
                  100.0.48.40        NaN
                  100.1.116.150      NaN
                                   ...  
iOS               99.82.254.180      NaN
                  99.89.44.176       NaN
                  99.92.172.96     133.0
                  99.93.208.84       NaN
                  Users              NaN
Length: 4044, dtype: float64